In [0]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib
import scipy

In [0]:
from matplotlib import pyplot as plt
matplotlib.style.use('seaborn')

In [0]:
# Data utilities
def index_by_date(df):
    try:
        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    except:
        df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
    df = df.set_index('date')
    
    return df

def adjust_returns(spot_return, decay):
    spot_momentum = spot_return.ewm(com=decay / (1.0 - decay), adjust=False).mean()
    adj_return = spot_return - spot_momentum
    
    return adj_return

In [0]:
path = '/content/'
usd_ret = pd.read_csv('{}usd_returns.csv'.format(path))
chf_ret = pd.read_csv('{}chf_returns.csv'.format(path))
premia = pd.read_csv('{}premia_returns.csv'.format(path))
usd_ret = index_by_date(usd_ret)
chf_ret = index_by_date(chf_ret)
premia = index_by_date(premia)

In [0]:
# Here you could select only the ccy pairs you want in the portfolio, usd or chf base
start_sim = '2008-01-01'
portfolio_returns = usd_ret.copy()
portfolio_returns['target'] =  premia['target']
portfolio_returns = portfolio_returns[start_sim:]

In [39]:
portfolio_returns.head()

,AUDUSD,EURUSD,GBPUSD,NZDUSD,CADUSD,CHFUSD,JPYUSD,NOKUSD,SEKUSD,target
date,,,,,,,,,,
2008-01-01,0.001906,0.001691,0.002161,0.000042,-0.005947,0.003084,0.003162,0.000562,0.003022,-0.000776
2008-01-02,0.002467,0.008436,-0.003382,0.008366,0.006531,0.014324,0.018418,0.007301,0.007972,-0.000891
2008-01-03,0.001785,0.000666,-0.002535,0.003269,0.002218,0.002788,-0.002068,0.009821,0.007410,-0.003306
2008-01-04,-0.008456,0.001686,-0.001524,-0.007743,-0.012973,0.004964,0.009203,0.002504,0.001890,-0.000407
2008-01-07,-0.003232,-0.005438,-0.000355,-0.001786,-0.003291,-0.009693,-0.005382,-0.007994,-0.007615,0.002526


In [0]:
# Constants
decay = 0.97
w_max = 1
w_min = -1
spars_param = 1 / 1000000

In [0]:
port_adjRet = adjust_returns(portfolio_returns, decay)
cov_matr_arr = port_adjRet.ewm(com=decay / (1.0 - decay), adjust=False).cov(bias=True)
m, n = len(cov_matr_arr.index.levels[0]), len(cov_matr_arr.index.levels[1])
cov_matr_arr = cov_matr_arr.values.reshape(m, n, -1).swapaxes(1, 2)
time = 3000
covMat = cov_matr_arr[time]

Sean

In [0]:
newcov = covMat[:9,:9]
q = covMat[9][:9]

Usaré la notación de http://web.stanford.edu/~boyd/papers/pdf/socp.pdf p.5

In [0]:
p_0sqrt = scipy.linalg.sqrtm(newcov)
p_0minussqrt = np.linalg.matrix_power(p_0sqrt,-1)
q_0 = -q

In [0]:
f = np.array([0,0,0,0,0,0,0,0,0])  #esta f es t, pero ya había usado t antes

In [0]:
#f = np.random.randn(9)


In [0]:
x = cp.Variable(9)
soc_constraints = [
      cp.SOC(f.T @ x, p_0sqrt @ x + np.matmul(p_0minussqrt,q_0))]
prob = cp.Problem(cp.Minimize(f.T@x), soc_constraints)

In [299]:
prob.solve()

0.0

In [300]:
x.value

array([-0.00477405,  0.18379647,  0.0764348 ,  0.07511024,  0.05619531,
       -0.09242163,  0.02759113, -0.23144184, -0.06578998])

es lo mismo que:

In [0]:
size = 9
x = cp.Variable(size)
a = np.identity(size)
constraints = []
objective = cp.Minimize(cp.quad_form(x, newcov) - 2*q.T @ x)
prob = cp.Problem(objective, constraints)

In [257]:
prob.solve()

-7.236487671237384e-07

In [258]:
x.value

array([-0.00477407,  0.18379644,  0.07643478,  0.07511021,  0.05619529,
       -0.09242166,  0.02759111, -0.2314419 , -0.06579002])

la nuevas condiciones se escriben como $(x_i\Sigma') x - 0.4 x^T \Sigma x \leq 0$ donde $\Sigma$ es la matriz de covarianza de las 9 currencies ie *newcov* y $\Sigma'$ es el producto punto de la fila $i$ de $\Sigma$ con el vector $x$ de pesos. Además $x_i$ es la coordenada $i$ de $x$ que se obtiene como $a[i] \cdot x$ (producto punto) donde $a[i]$ es la fila $i$-ésima de la matriz identidad ($I_9$) 

Usando la notación de http://web.stanford.edu/~boyd/papers/pdf/socp.pdf p.5, tendríamos 9 constraints de la forma

$x^T P_i x + 2 q_i^T X \leq 0 $

con $i = 1, \ldots, 9$, así que $P_i = \sqrt(0.4 \Sigma)$ (esto es igual para todas) y $q_i^T = (1/2) x_i \Sigma' $ por ej la constraint $i=5$ se escribe así: 

In [0]:
a = np.identity(9)

In [322]:
p_5sqrt = scipy.linalg.sqrtm(newcov)

SyntaxError: ignored

In [0]:
sigma_prime = newcov[5]
q_5 = 1/2*a[5]*sigma_prime

el cuadrado de $P_5$ es la sig matriz

In [0]:
p_5sqrt = scipy.linalg.sqrtm(4*newcov)

In [0]:
p_5minussqrt = np.linalg.matrix_power(p_5sqrt,-1)


In [0]:
p_5totheminusone = np.linalg.matrix_power(p_5sqrt,-2)

In [0]:
q_5trans_P_5minusone_q5 = np.sqrt(np.matmul(np.transpose(q_5),np.matmul(p_5totheminusone, q_5)))

In [319]:
x = cp.Variable(9)
soc_constraints = [
      cp.SOC(f.T @ x, p_0sqrt @ x + np.matmul(p_0minussqrt,q_0)),
      cp.SOC(q_5trans_P_5minusone_q5, p_5sqrt @ x + np.matmul(p_5minussqrt,q_5))]
prob = cp.Problem(cp.Minimize(f.T@x), soc_constraints)

AttributeError: ignored